In [ ]:
import json
import os

import matplotlib.pyplot as plt
import sagemaker
import numpy as np

from sagemaker.analytics import HyperparameterTuningJobAnalytics

from source.visualization.model_visualisation_utils import get_dfs_from_hpt, plot_df_list, get_best_training_job

In [ ]:
from source.config import Config
config = Config(filename="config/config.yaml")

# Get the data from your Hyperparameter tuning job
Enter your HPO job name to visualise the data

In [ ]:
hpo_job_name = config.hpo_job_name

hpt = HyperparameterTuningJobAnalytics(hpo_job_name)

In [ ]:
summaries = hpt.training_job_summaries()
dfs = get_dfs_from_hpt(summaries, metrics=["Epoch", "train_auc", "train_acc", "train_loss",
                                           "test_auc", "test_acc", "test_loss"])

# Visualise all the jobs

In [ ]:
plot_df_list(dfs, metric_name="test_acc", y_label="Test accuracy", min_final_value=0.70)

# Visualise the data from the best training job

In [ ]:
best_job_name, best_job_df = get_best_training_job(dfs, "test_auc", "maximize")
best_job_df.head()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(1500/108, 400/108), dpi=108)

axs[0].plot(best_job_df["Epoch"], best_job_df["test_loss"], label=best_job_name)
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("Test loss")
axs[0].grid(color="0.9", linestyle='-', linewidth=3)

axs[1].plot(best_job_df["Epoch"], best_job_df["test_auc"], label=best_job_name)
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Test AUC")
axs[1].grid(color="0.9", linestyle='-', linewidth=3)

axs[2].plot(best_job_df["Epoch"], best_job_df["test_acc"], label=best_job_name)
axs[2].set_xlabel("Epochs")
axs[2].set_ylabel("Test Accuracy")
axs[2].grid(color="0.9", linestyle='-', linewidth=3)

plt.tight_layout()
plt.show()

# Build endpoint for the best training job

In [ ]:
sage_session = sagemaker.session.Session()
model_artifact = sage_session.describe_training_job(best_job_name)["ModelArtifacts"]["S3ModelArtifacts"]
model_artifact = os.path.join(os.path.dirname(model_artifact), "output.tar.gz")
print("Building endpoint with model {}".format(model_artifact))

In [ ]:
from sagemaker.pytorch import PyTorchModel
role = sagemaker.get_execution_role()


model = PyTorchModel(model_data=model_artifact,
                     role=role,
                     entry_point="inference.py",
                     source_dir="source/dl_utils",
                     framework_version='1.5.0',
                     name="fpm-model"
                    )

endpoint_instance_type = "ml.t2.medium"

predictor = model.deploy(initial_instance_count=1, instance_type=endpoint_instance_type, endpoint_name='fpm-endpoint')

def custom_np_serializer(data):
    return json.dumps(data.tolist())
    
def custom_np_deserializer(np_bytes, content_type='application/x-npy'):
    out = np.array(json.loads(np_bytes.read()))
    return out

predictor.serializer = custom_np_serializer
predictor.deserializer = custom_np_deserializer

## Test the predictor

In [ ]:
data = np.ones(shape=(1, 20, 2))
out = predictor.predict(data)
print("The predicted output is prob of no failure is {:0.4f}".format(out[0, 0])) 